In [2]:
import pandas as pd
import pickle as pkl
# POT needs to be installed for the following code to work
# !pip install POT

df = pd.read_csv('data/children-with-query-frequencies-and-tokens.txt', sep='\t')

frequencies = pd.read_csv('data/children-query-frequencies-precomputed.txt', sep='\t', index_col=0)
tokens = pd.read_csv('data/children-query-tokens-precomputed.txt', sep='\t', index_col=0)
idf_scores = pd.read_csv('data/children-idf-scores-precomputed.txt', sep='\t', index_col=0)

# Dataset statistics
print('Number of rows:', df.shape[0])
# print('Number of unique users:', df['AnonID'].nunique())
print('Number of unique queries:', df['Query'].nunique())
print('Query frequencies range:  [{}, {}]'.format(df['QueryFrequency'].min(), df['QueryFrequency'].max()))
print('Average query length:', df['Query'].apply(len).mean())

# User 479 queries:
# df[df['AnonID'] == 479]

Number of rows: 603
Number of unique queries: 596
Query frequencies range:  [1, 4]
Average query length: 21.432835820895523


In [3]:
# Most frequent users and their query counts
# print("Top 10 users by query count")
# print(df['AnonID'].value_counts().head(10))

# print("-" * 50)

# Most frequent queries
print("Top 10 queries by frequency")
print(df['Query'].value_counts().head(10))

Top 10 queries by frequency
Query
youtube                             4
games                               3
Star wars                           2
weather                             2
best walking shoes for babies       1
best gps deals                      1
best love novels                    1
best professional digital camera    1
best rc truck                       1
best songs of the 80s and 90s       1
Name: count, dtype: int64


In [5]:
# Query completion, based on the most frequent queries starting with the given query
def query_completion(query):
    starts_with_query = df['Query'].str.startswith(query)

    return df[starts_with_query].sort_values('QueryFrequency', ascending=False)['Query'].unique()[:10]

print("10 top completions for 'fast':")
pd.DataFrame(query_completion('fast'), columns=[''])

10 top completions for 'fast':


,
0,fast in the fouris
1,fast food design
2,fast now


### Query Auto-Completion Based on Word2vec Semantic Similarity
Considers semantic similarity between the candidate queries and their previous queries submitted in the same session, on the basis of word2vec method.


In [7]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tqdm import tqdm  # Progress bar for training

# Tokenize queries (with tqdm progress bar)
queries = df['Query']
query_tokens = df['QueryTokens']

# Initialize word2vec model
AOL_model = Word2Vec(vector_size=200,   # Dimension of the word vectors
                 window=5,          # Context window size
                 min_count=1,       # Minimum word frequency
                 workers=4,         # Number of parallel workers
                 sg=1)              # Skip-gram model

# Train word2vec model
AOL_model.build_vocab(tqdm(query_tokens, desc='Building word2vec vocabulary'))

AOL_model.train(tqdm(query_tokens, desc='Training word2vec model'),
            total_examples=AOL_model.corpus_count,
            epochs=30)

# Save word2vec model
AOL_model.save('word2vec_AOL.model')

Training word2vec model: 100%|██████████| 603/603 [00:00<00:00, 119996.46it/s]


In [8]:
from gensim.models import KeyedVectors
import gensim.downloader as api

# Load Google's pre-trained Word2Vec model
# google_model = KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin', binary=True)  # if you have it locally
google_model = api.load('word2vec-google-news-300')

# Load the trained AOL word2vec model
# AOL_model = Word2Vec.load('models/word2vec_AOL.model')   # Only if you don't want to retrain it (but it's quite fast to train, ~15 min)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


MemoryError: Unable to allocate 3.35 GiB for an array with shape (3000000, 300) and data type float32

In [6]:
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
import numpy as np

# Query auto-completion function
def query_completion(query, completion_list, session_queries, alpha=0.6):
    N = len(session_queries)
    gamma = beta = 1/(N+1)
    omega = 0.5

    # Frequency score (of each candidate query)
    frequency_score = [frequencies.loc[a, 'Frequency'] if a in frequencies.index else 0 for a in completion_list]

    # Semantic similarity score
    # Use average similarity score on words (average of all word pairs)
    similarity_score = []
    for candidate_query in completion_list:  # Possible completions
        candidate_score = 0

        for session_query in session_queries:  # User session queries
            google_model_score = 0
            AOL_model_score = 0

            c_tokens = word_tokenize(candidate_query)
            x_tokens = word_tokenize(session_query)

            # Add similarities of all word pairs between the candidate query and the session query,
            # weighted by the idf of the words
            for c in c_tokens:
                for x in x_tokens:
                    # Check if the words are in the models' vocabularies
                    if c in google_model.key_to_index and x in google_model.key_to_index:
                        google_model_similarity = google_model.similarity(c, x)
                    else:
                        google_model_similarity = 0
                    if c in AOL_model.wv.key_to_index and x in AOL_model.wv.key_to_index:
                        AOL_model_similarity = AOL_model.wv.similarity(c, x)
                    else:
                        AOL_model_similarity = 0

                    # Idf weighting average:
                    if c in idf_scores.index.values and x in idf_scores.index.values:
                        # Idf scores (precomputed)
                        c_idf = idf_scores.loc[idf_scores.index == c, 'IDF'].values[0]
                        x_idf = idf_scores.loc[idf_scores.index == x, 'IDF'].values[0]

                        google_model_score += (google_model_similarity * c_idf * x_idf)
                        AOL_model_score += (AOL_model_similarity * c_idf * x_idf)
                    else:
                        google_model_score += google_model_similarity
                        AOL_model_score += AOL_model_similarity

            # Average, divide by all combinations of words:
            google_model_score /= (len(c_tokens) * len(x_tokens))
            AOL_model_score /= (len(c_tokens) * len(x_tokens))

            # Some query words might not appear in the google model's vocabulary:
            if google_model_score == 0:
                candidate_score += AOL_model_score
            else:
                candidate_score += omega * AOL_model_score + (1 - omega) * google_model_score

        similarity_score.append(candidate_score)

    # Combined score (alpha * similarity_score + (1-alpha) * frequency_score)
    combined_score = [alpha * similarity_score[i] + (1 - alpha) * frequency_score[i] for i in range(len(completion_list))]

    # Re-rank completion list
    re_ranked_list = [x for _, x in sorted(zip(combined_score, completion_list), reverse=True)]

    return re_ranked_list

In [ ]:
# Dummy test the query auto-completion function
# query = 'car'
# completion_list = df[df['Query'].str.startswith(query)]['Query'].unique()  # Queries starting with 'car'
# session_queries = df[(df['AnonID'] == 479) & (df['Query'].str.contains('car'))]['Query'].unique()  # Queries from user 479 containing 'car'
# print(session_queries)

# print("Query auto-completion for 'car':")
# print(query_completion(query, completion_list, session_queries))

In [7]:
# MRR (Mean Reciprocal Rank) evaluation
def RR(ranked_completions, ground_truth):
    """
    Reciprocal Rank (RR) for one query.
    :param ranked_completions: list of suggested completions (higher rank first)
    :param ground_truth: the correct suggestion of the query
    :return: the RR score
    """
    for i, completion in enumerate(ranked_completions):
        if completion == ground_truth:
            return 1.0 / (i + 1)
    return 0.0

def MRR(completion_lists, ground_truths):
    """
    Mean of scores for entire dataset.
    """
    total_score = 0.0
    for i, completion_list in enumerate(completion_lists):
        total_score += RR(completion_list, ground_truths[i])

    return total_score / len(completion_lists)

In [8]:
# Train and test datasets (arrays of query sessions, sorted by time increasing)
with open('data/children-train.pkl', 'rb') as f:
    train = pkl.load(f)

with open('data/children-test.pkl', 'rb') as f:
    test = pkl.load(f)

# Flatten train dataset
train = [query for session in train for query in session]
train = pd.DataFrame(train, columns=['Query'])

print("Train dataset size: ", len(train), "queries")
print("Test dataset size: ", len(test), "sessions and", len([query for session in test for query in session]), "queries")
print("First 10 sessions in test dataset: ")
print(test[:10])
# Session lengths counts
print("Session lengths in test dataset: ")
pd.Series([len(session) for session in test]).value_counts()

Train dataset size:  2709614 queries
Test dataset size:  262547 sessions and 904763 queries
First 10 sessions in test dataset: 
[['-', '-', '-', '-'], ['myspace.com'], ['pet sitter in newburyport ma', 'pet sitter in newburyport ma'], ['undefined'], ['shakira lyrics'], ['ebay', 'social security'], ['glutes', 'glutes', 'glutes', 'glutes', 'glutes', 'adultfriendfinder'], ['sandals vacations'], ['www.delta.com'], ['costco']]
Session lengths in test dataset: 


1      113402
2       51078
3       28311
4       17455
5       11957
        ...  
122         1
138         1
183         1
162         1
150         1
Length: 141, dtype: int64

In [17]:
print("Word count of last session query in test dataset: ")
pd.Series([len(word_tokenize(session[-1])) for session in test]).value_counts()

Word count of last session query in test dataset: 


1     127437
2      59492
3      36285
4      19425
5       9875
       ...  
92         1
33         1
95         1
49         1
87         1
Length: 76, dtype: int64

In [10]:
def get_completions(sessions, prefix_length, train_df, alpha=0.6):
    completion_lists = []
    ground_truths = []

    for session in tqdm(sessions, desc='Generating completions'):
        # Last query in the session is the query we want to complete
        query_ground_truth = session[-1]
        query_tokenized = word_tokenize(query_ground_truth)

        # Skip if the query is shorter than the prefix length
        if len(query_tokenized) < prefix_length:
            continue

        # Queries preceding the last query in the session
        previous_queries = session[:-1] if len(session) > 1 else []

        # Get prefix (part of the query that needs to be completed)
        query_prefix = ' '.join(query_tokenized[:prefix_length])

        # Get completions
        completions = train_df[train_df['Query'].str.startswith(query_prefix + " ")]['Query'].unique() # here we add a space after the word so we don't look for partial matches

        # Re-rank completions
        ranked_completions = query_completion(query_prefix, completions, previous_queries, alpha)

        completion_lists.append(ranked_completions)
        ground_truths.append(query_ground_truth)

    return completion_lists, ground_truths

def evaluate_test_set(test, prefix_length, alpha=0.6):
    completion_lists, ground_truths = get_completions(test, prefix_length, train, alpha)
    mrr = MRR(completion_lists, ground_truths)
    return mrr

In [11]:
# Evaluate the test set
prefix_length = 5   # query will be completed based on the first 3 words
mrr = evaluate_test_set(test, prefix_length)
print("MRR for prefix length", prefix_length, ":", mrr)

Generating completions:   0%|          | 341/262547 [00:25<5:31:38, 13.18it/s] 


KeyboardInterrupt: 

In [ ]:
# Evaluate the test set
prefix_length = 4   # query will be completed based on the first 3 words
mrr = evaluate_test_set(test, prefix_length)
print("MRR for prefix length", prefix_length, ":", mrr)

In [12]:
# Evaluate the test set
prefix_length = 3   # query will be completed based on the first 3 words
mrr = evaluate_test_set(test, prefix_length)
print("MRR for prefix length", prefix_length, ":", mrr)

Generating completions: 100%|██████████| 20/20 [01:15<00:00,  3.77s/it]

MRR for prefix length 3 : 0.14285714285714285


In [ ]:
# Evaluate the test set
prefix_length = 2   # query will be completed based on the first 2 words
mrr = evaluate_test_set(test, prefix_length)
print("MRR for prefix length", prefix_length, ":", mrr)

In [ ]:
# Evaluate the test set
prefix_length = 1   # query will be completed based on the first word (VERY SLOW)
mrr = evaluate_test_set(test, prefix_length)
print("MRR for prefix length", prefix_length, ":", mrr)